# Hypothesis Testing to Identify Key Predictors of the Model (As part of EDA)

## Identifying Key predictors from T-Test and Chi-Square Test

In [25]:
#import libraries
import pandas as pd
import numpy as np

In [27]:
#loading data
df = pd.read_csv(r"C:\Users\Pulani\Desktop\NTUCLHUB\MYPROJECT_CAPSTONE\Project 1 - Finance - Predictive Analytics\Dataset\loan.csv")
print ("Data loaded successfully!")

# Display the first few rows of the dataset
df.head()

Data loaded successfully!


,customer_id,loan_id,loan_type,loan_amount,interest_rate,loan_term,employment_type,income_level,credit_score,gender,marital_status,education_level,application_date,approval_date,disbursement_date,due_date,default_status
0,CUST-00004912,LN00004170,Car Loan,16795,0.051852,15,Self-employed,Medium,833,Male,Single,Master,05-04-2018,23-04-2018,24-04-2018,14-08-2018,False
1,CUST-00004194,LN00002413,Personal Loan,1860,0.089296,56,Full-time,Medium,776,Female,Married,Bachelor,30-12-2022,31-12-2022,12-01-2023,05-04-2023,False
2,CUST-00003610,LN00000024,Personal Loan,77820,0.070470,51,Full-time,Low,697,Male,Divorced,High School,15-11-2019,18-11-2019,27-11-2019,24-02-2020,False
3,CUST-00001895,LN00001742,Car Loan,55886,0.062155,30,Full-time,Low,795,Female,Married,PhD,25-08-2021,08-09-2021,11-09-2021,25-02-2022,False
4,CUST-00003782,LN00003161,Home Loan,7265,0.070635,48,Part-time,Low,519,Female,Married,High School,02-09-2020,07-09-2020,11-09-2020,29-12-2020,False


In [29]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency

### T-Test
**To check whether there's a significant difference between the means of two groups**, such as the difference in loan_amount for defaulters (default_status = 1) vs non-defaulters (default_status = 0).

In [32]:
#T-test

import pandas as pd
import numpy as np
from scipy import stats

# Numerical columns to be considered
numerical_cols = ['loan_amount', 'interest_rate', 'loan_term', 'credit_score']

# Loop through each numerical column and apply T-test
for col in numerical_cols:
    group1 = df[df['default_status'] == 1][col]  # Defaulters
    group2 = df[df['default_status'] == 0][col]  # Non-defaulters
    
    t_stat, p_value = stats.ttest_ind(group1, group2)
    
    print(f"\nT-test for {col}:")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The difference in {col} between defaulters and non-defaulters is significant.")
    else:
        print(f"The difference in {col} between defaulters and non-defaulters is not significant.")


T-test for loan_amount:
T-statistic: -0.5167081777542388, P-value: 0.6053827484468421
The difference in loan_amount between defaulters and non-defaulters is not significant.

T-test for interest_rate:
T-statistic: 2.048444981671618, P-value: 0.04056854478895377
The difference in interest_rate between defaulters and non-defaulters is significant.

T-test for loan_term:
T-statistic: -0.8737435528564498, P-value: 0.3822999056713817
The difference in loan_term between defaulters and non-defaulters is not significant.

T-test for credit_score:
T-statistic: -0.5193623806018853, P-value: 0.6035310574806859
The difference in credit_score between defaulters and non-defaulters is not significant.


### T-Test for date column
**For this purpose date collumns should be converted in to numerical column**

In [42]:
#convert date columns to numeric

# Set the reference date to January 1, 1970 (Unix Epoch)
reference_date = pd.to_datetime('1970-01-01')

# Convert date columns to datetime format, with day first
df['application_date'] = pd.to_datetime(df['application_date'], dayfirst=True)
df['approval_date'] = pd.to_datetime(df['approval_date'], dayfirst=True)
df['disbursement_date'] = pd.to_datetime(df['disbursement_date'], dayfirst=True)
df['due_date'] = pd.to_datetime(df['due_date'], dayfirst=True)

# Calculate the number of days since the reference date (Unix Epoch)
df['application_days'] = (df['application_date'] - reference_date).dt.days
df['approval_days'] = (df['approval_date'] - reference_date).dt.days
df['disbursement_days'] = (df['disbursement_date'] - reference_date).dt.days
df['due_days'] = (df['due_date'] - reference_date).dt.days

# Drop the original date columns
df = df.drop(['application_date', 'approval_date', 'disbursement_date', 'due_date'], axis=1)

# Check the updated DataFrame
print("Updated DataFrame after processing date columns:")
df.head()

Updated DataFrame after processing date columns:


,customer_id,loan_id,loan_type,loan_amount,interest_rate,loan_term,employment_type,income_level,credit_score,gender,marital_status,education_level,default_status,application_days,approval_days,disbursement_days,due_days
0,CUST-00004912,LN00004170,Car Loan,16795,0.051852,15,Self-employed,Medium,833,Male,Single,Master,False,17626,17644,17645,17757
1,CUST-00004194,LN00002413,Personal Loan,1860,0.089296,56,Full-time,Medium,776,Female,Married,Bachelor,False,19356,19357,19369,19452
2,CUST-00003610,LN00000024,Personal Loan,77820,0.070470,51,Full-time,Low,697,Male,Divorced,High School,False,18215,18218,18227,18316
3,CUST-00001895,LN00001742,Car Loan,55886,0.062155,30,Full-time,Low,795,Female,Married,PhD,False,18864,18878,18881,19048
4,CUST-00003782,LN00003161,Home Loan,7265,0.070635,48,Part-time,Low,519,Female,Married,High School,False,18507,18512,18516,18625


In [44]:
#T-test to date column

from scipy import stats

# Perform T-test for the transformed date column 'application_days'
group1 = df[df['default_status'] == 1]['application_days']  # Defaulters
group2 = df[df['default_status'] == 0]['application_days']  # Non-defaulters

t_stat, p_value = stats.ttest_ind(group1, group2)

print(f"T-test for 'application_days': T-statistic = {t_stat}, P-value = {p_value}")

if p_value < 0.05:
    print("The difference in 'application_days' between defaulters and non-defaulters is significant.")
else:
    print("The difference in 'application_days' between defaulters and non-defaulters is not significant.")

T-test for 'application_days': T-statistic = -2.2155895891171307, P-value = 0.026764521013511674
The difference in 'application_days' between defaulters and non-defaulters is significant.


**Summary of Findings from T-Test**:

- Loan Amount:
T-statistic: -0.517
P-value: 0.605
Conclusion: The difference in loan_amount between defaulters and non-defaulters is not significant. This means the loan amounts for defaulters and non-defaulters are not statistically different.

- Interest Rate:
T-statistic: 2.048
P-value: 0.041
Conclusion: The difference in interest_rate between defaulters and non-defaulters is significant. This suggests that interest rates for defaulters and non-defaulters are statistically different, with a p-value below 0.05.

- Loan Term:
T-statistic: -0.874
P-value: 0.382
Conclusion: The difference in loan_term between defaulters and non-defaulters is not significant. Loan terms are similar across defaulters and non-defaulters.

- Credit Score:
T-statistic: -0.519
P-value: 0.604
Conclusion: The difference in credit_score between defaulters and non-defaulters is not significant. Credit scores do not differ significantly between the two groups.

- **Actionable Insights:**
- **Interest Rate** is the only numerical feature where there is a significant difference between defaulters and non-defaulters. This could be an important feature to focus on in the predictive model.
- **application_days** is significantly different between defaulters and non-defaulters, indicating that it might be an important predictor for your model.


The other features (loan_amount, loan_term, credit_score) do not show significant differences between defaulters and non-defaulters, but I may still include them in your model as they could interact with other variables in a non-linear way.

### Chi-Square Test 
**To check if there's an association between two categorical variables**, such as employment_type and default_status.

In [38]:
#Chi-Square

import pandas as pd
from scipy.stats import chi2_contingency

# List of categorical columns
categorical_cols = ['loan_type', 'employment_type', 'income_level', 'gender', 'marital_status', 'education_level']

# Loop through each categorical column and perform the Chi-Square test with `default_status`
for col in categorical_cols:
    # Create a contingency table for the categorical feature and default_status
    contingency_table = pd.crosstab(df[col], df['default_status'])
    
    # Perform Chi-Square test
    chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
    
    # Display the results
    print(f"\nChi-Square Test for {col}:")
    print(f"Chi-Square Statistic: {chi2_stat}")
    print(f"P-value: {p_val}")
    
    # Interpretation
    if p_val < 0.05:
        print(f"There is a significant association between {col} and default_status.")
    else:
        print(f"There is no significant association between {col} and default_status.")


Chi-Square Test for loan_type:
Chi-Square Statistic: 8.583853598615384
P-value: 0.0353673439978112
There is a significant association between loan_type and default_status.

Chi-Square Test for employment_type:
Chi-Square Statistic: 3.0594530311843577
P-value: 0.21659489454324748
There is no significant association between employment_type and default_status.

Chi-Square Test for income_level:
Chi-Square Statistic: 0.2452432595494604
P-value: 0.8845983049073796
There is no significant association between income_level and default_status.

Chi-Square Test for gender:
Chi-Square Statistic: 7.460744316882963
P-value: 0.0063058927844888285
There is a significant association between gender and default_status.

Chi-Square Test for marital_status:
Chi-Square Statistic: 1.1767039953034573
P-value: 0.5552415705671334
There is no significant association between marital_status and default_status.

Chi-Square Test for education_level:
Chi-Square Statistic: 1.724708414485796
P-value: 0.63145459318359

**Summary of Chi-Square Test Results:**

- Loan Type:

Chi-Square Statistic: 8.584
P-value: 0.035
Conclusion: There is a significant association between loan_type and default_status, meaning the loan type is significantly related to whether a customer defaults or not.

- Employment Type:
Chi-Square Statistic: 3.059
P-value: 0.217
Conclusion: There is no significant association between employment_type and default_status, indicating that employment type does not play a significant role in determining default status.

- Income Level:
Chi-Square Statistic: 0.245
P-value: 0.885
Conclusion: There is no significant association between income_level and default_status.

- Gender:
Chi-Square Statistic: 7.461
P-value: 0.006
Conclusion: There is a significant association between gender and default_status, suggesting that gender may play a role in default likelihood.

- Marital Status:
Chi-Square Statistic: 1.177
P-value: 0.555
Conclusion: There is no significant association between marital_status and default_status.

- Education Level:
Chi-Square Statistic: 1.725
P-value: 0.631
Conclusion: There is no significant association between education_level and default_status.

**Key Insights:**
 
**Significant Associations: loan_type and gender** are significantly associated with default_status. These features could be important predictors in your model.

No Significant Association: employment_type, income_level, marital_status, and education_level do not show significant relationships with default status and may not be as useful for predicting defaults.

## Summary of Insights:

**Imbalanced Dataset**: The original dataset is highly imbalanced (many more non-defaulters than defaulters).

**Significant Features**: - **Interest Rate, application_days** is significantly different between defaulters and non-defaulters, 
                            indicating that it might be an important predictor to this model.
                            - **Loan Type, Gender** are Significant predictors based on Chi-square analysis.
